In [ ]:
# Install a pip package in the current Jupyter kernel
import time
import sys
!{sys.executable} -m pip install RPi.GPIO

In [ ]:
from Raspi_MotorHAT import Raspi_MotorHAT
motor = Raspi_MotorHAT(addr=0x6f, freq=250).getMotor(1)

def motor_brk():
    global motor
    # не понятно почему этот метод не заимплеменчен в Raspi_DCMotor
    motor.MC.setPin(motor.IN1pin, 1)
    motor.MC.setPin(motor.IN2pin, 1)

def motor_fwd(speed):
    global motor
    motor.setSpeed(speed)
    motor.run(Raspi_MotorHAT.FORWARD)

def motor_bac(speed):
    global motor
    motor.setSpeed(speed)
    motor.run(Raspi_MotorHAT.BACKWARD)
    
def motor_rel():
    global motor
    motor.run(Raspi_MotorHAT.RELEASE)

In [ ]:
import RPi.GPIO as GPIO
import time 

# GPIO.cleanup()

GPIO.setmode(GPIO.BOARD)
TRIG = 16
ECHO = 18

GPIO.setup(TRIG, GPIO.OUT, initial=0)
GPIO.setup(ECHO, GPIO.IN)

try:
    while True:
        # Minimum delay between measurements is 50ms. 100ms is fine:
        time.sleep(0.1)

        # Send a probe signal to the sensor. Signal should be 10 micro sec long.
        GPIO.output(TRIG,1)
        time.sleep(0.00001)
        GPIO.output(TRIG,0)

        while GPIO.input(ECHO) == 0:
            pass 
            start = time.time()

        while GPIO.input(ECHO) == 1:
            pass 
            stop = time.time()
        
        minSpeed = 25
        maxSpeed = 100
        maxDistance = 200 # sm
        # Print distance to object in santimeters. Sound speed = 340 m/s
        distance = int((stop - start) * 17000);
        speed = int(distance / maxDistance) * maxSpeed;
        
        if speed > maxSpeed:
            speed = maxSpeed
        if speed < minSpeed:
            speed = minSpeed
            
        print(f"Distance = {distance} sm, speed {speed}")
        motor_fwd(speed)

except KeyboardInterrupt:
    motor_rel()
    GPIO.cleanup()